In [1]:
from ccnet_spark.text_normalizer import normalize
from ccnet_spark.pipe_preprocess import load_segments
from ccnet_spark.pipe_hash import compute_hashes,split_doc2para
from ccnet_spark.pipe_lid import predictLang
from ccnet_spark.pipe_tokenized import doSentencePiece
from ccnet_spark.pipe_perplexity import doDocLM
from ccnet_spark.pipe_ppbucket import doPPBucket
from ccnet_spark.pipe_save import save_partation,load_partation,load_all
import time
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import explode
from pyspark.sql.functions import sum as spark_sum

# 初始化 SparkSession
spark = SparkSession.builder.appName("CCNETSpark")  \
                    .config("spark.executor.memory", "64g") \
                    .config("spark.driver.memory", "32g") \
                    .config("spark.driver.maxResultSize", "32g") \
                    .config('spark.sql.execution.arrow.pyspark.enabled', 'true') \
                    .getOrCreate()

24/04/03 11:05:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
def getModePara(mode):
    if(mode=="test"):
        n_segments=10
        cache_folder="/root/wxl_folder/cache_data/"
        date="2019-09" ## hardcode ,现在只能是这个
        segments=[i for i in range(n_segments)]
        min_len=300
        isSample=True
        sampleRate=0.01
        num_partitions=1
    else:
        n_segments=4
        cache_folder="/root/wxl_folder/cache_data/"
        date="2019-09" ## hardcode ,现在只能是这个
        segments=[i for i in range(n_segments)]
        min_len=300
        isSample=False
        sampleRate=1
        num_partitions=4
    return [cache_folder,date,segments,min_len,isSample,sampleRate,num_partitions]
mode="dev"
cache_folder,date,segments,min_len,isSample,sampleRate,num_partitions=getModePara(mode)

In [45]:
selected_bucket="head"
selected_lang="zh"
df_p=load_partation(spark,selected_lang,selected_bucket,cache_folder,date,isSample,sampleRate,min_len)
res=df_p.filter(df_p.cc_segment == "1").select("url").collect()
sorted_res = sorted(res, key=lambda x: x.url)
print(len(sorted_res))
for url in sorted_res:
    print(url.url)

242
http://042699.com/56020844/
http://101pd.com/12640745/
http://105670.com/05905720/
http://105670.com/349444/
http://3kirikou.org/recommend_detail.php?SerialNo=1814&sh_subcategory=21&sh_author=&sh_tag=all&sh_startdate=&sh_enddate=&sh_keyword=&filter=tbRSN
http://8-nana.com/tw/contact/index.aspx
http://94yule.com/a/xinwenzixun/32.html
http://9918626.cn/YDiqc.html
http://a18.d72d.com/index.phtml?PUT=a_show&AID=87885&FID=375351&R2=&CHANNEL=
http://admission.blcu.edu.cn/131/list.htm
http://aehughes.net/html/html/2018/redu_0806/324370.html
http://ahshuhua.net/guohua/huiyuan/qishaolong2015c.html
http://alkwz.com/47614/649.html
http://anceguoji.com/tesco_view.php?classid=26&id=295
http://bali.fuzhou61.com/bali-jing-dian/40.html
http://beijing.qianlong.com/2016/0218/372094.shtml
http://bibles.org/zho-CUNP/Num/15/16
http://bjtds.com.cn/plus/list.php?tid=31
http://bjyucai.com/news/news_detail.asp?id=3846
http://blkjck.pixnet.net/blog/post/41315539-%E6%83%85%E4%BA%BA%E7%AF%80%E5%BF%AB%E6%A8%82

In [5]:
df=load_all(spark,cache_folder,date,isSample,sampleRate,min_len)

In [46]:
df.filter(df.url=="http://7339251.com/a/yulewanzhengban/20190112/57452.html").head()

Row(digest='sha1:LALGNXVRAQVDQ3YKF3V7BOKVHJ2PR7WD', url='http://7339251.com/a/yulewanzhengban/20190112/57452.html', date_download='2019-02-15T19:16:32Z', source_domain='7339251.com', cc_segment='1', original_length=2711, original_nlines=102, title='【图】14年前的《泰坦尼克号》 50块电影票炒到200块_青娱乐', nlines=64, raw_content='青娱乐 - 青娱乐视频，青娱乐在线！\n高级搜索|网站地图|TAG标签RSS订阅[设为首页] [加入收藏]\n青娱乐\n青娱乐美女\n青娱乐视频\n青娱乐在线\n热门标签:\n播出\n荧屏\n要\n娱乐圈\n科比\n杨幂\n当前位置: 主页 > 青娱乐在线 >\n【图】14年前的《泰坦尼克号》 50块电影票炒到200块\n时间:2019-01-12 21:26来源:网络整理 作者:青娱乐 点击: 次\n【图】14年前的《泰坦尼克号》 50块电影票炒到200块 data.dkeys\n14年前的报纸\n●人均月收入不到1200块，一张黄牛票炒到200块\n●影院门口看自行车的，一天收入600块\n●为了抢拷贝，省公司和市公司的人打了起来\n全国1.5亿，上海1500万？孙航宇一听这数字头就大了。\n孙航宇是上海市电影公司的副总经理，当时他和全国各省电影公司的发行经理们正在北京的中影总公司开碰头会，商讨一部预计在当年4月份上映的引进大片各省必须完成的保底票房数字。上海市作为票房重镇分到了全国1.5亿保底票房的1/10，也就是1500万人民币。\n这个数字现在听起来并不高，但“当时”是1998年，之前的1997年贺岁大片《甲方乙方》全 国“大卖”，也不过3000多万；历史最高票房纪录保持者是施瓦辛格的《真实谎言》———1.2亿。这部电影长达3个小时，90分钟的电影一天能排8场， 这部电影最多只能排5场。而且，一个中国观众脸熟的明星都没有！中影开出的数字简直是痴心妄想！以孙航宇为代表的发行经理们都不肯接受中影的保底数字，这 个会从上午一直开到下午，孙航宇一直“据理力争”